## This Program predicts the chances of an employee leaving the code based on his behaviour

In [2]:
from datetime import datetime
import time
from collections import namedtuple
import seaborn as sns; 
sns.set(style="ticks", color_codes=True)
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import precision_recall_fscore_support
now = datetime.utcnow(). strftime("% Y% m% d% H% M% S")
root_logdir = "tf_logs"
logdir = "{}/ run-{}/". format( root_logdir, now)

###### Open the HR dataset from Kaggle

In [3]:
#There is a werid issue here, by default the dataset is very very skewed and the Accuracy is around 70%, but
# if I decrease the skew, I am getting great resuts - Almost 95+ accuracy 

In [4]:
df = pd.read_csv('HR_comma_sep.csv')
# left_df = df[df.left == 1]#[:100]
# stay_df = df[df.left == 0][:int(len(left_df)*1.00)]
# df = pd.concat([left_df,stay_df])
df = shuffle(shuffle(df))
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
9855,0.67,0.83,3,171,3,1,0,0,marketing,medium
4166,0.70,0.74,3,157,3,0,0,0,technical,high
1753,0.91,1.00,4,245,6,0,1,0,RandD,medium
3775,0.70,0.71,5,269,3,0,0,0,technical,low
2998,0.53,0.53,4,205,3,0,0,0,sales,low


###### Exploratory Data Analysis

In [5]:
# #Display Feature Relevance
# %matplotlib inline 
# #g = sns.FacetGrid(df, col="left")
# #g = g.map(plt.hist, "average_montly_hours" )
# g = sns.PairGrid(df,hue="left")
# g.map_diag(plt.hist)
# g.map_offdiag(plt.scatter)
# g.add_legend();

###### Normalize all the values by using min max

In [6]:
df.left.describe()

count    14999.000000
mean         0.238083
std          0.425924
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: left, dtype: float64

In [7]:
# This function returns the normalized df
def one_hot_encode(x, n_classes):
    return np.eye(n_classes)[x.reshape(-1)]

def normalize_attributes(df):
    return df/max(df)
#     mu = np.mean(df,axis=0)
#     sigma = np.std(df,axis=0)
#     return (df - mu)/sigma

###### Pick up the interested features and targets

In [8]:

df["last_evaluation_norm"] = normalize_attributes(df.last_evaluation)
df["satisfaction_level_norm"] = normalize_attributes(df.satisfaction_level)
df["number_project_norm"]= normalize_attributes(df.number_project)
df["average_montly_hours_norm"] = normalize_attributes(df.average_montly_hours)
df["time_spend_company_norm"] = normalize_attributes(df.time_spend_company)
df["salary_norm"] = df.salary.apply(lambda x: 1.0 if x == "high" else 0.5 if x == "medium" else 0.0)
df["dept_norm"] = normalize_attributes(df.sales.astype('category').cat.codes)
df["left_norm"] = normalize_attributes(df.left)

In [9]:
df.head(10)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,last_evaluation_norm,satisfaction_level_norm,number_project_norm,average_montly_hours_norm,time_spend_company_norm,salary_norm,dept_norm,left_norm
9855,0.67,0.83,3,171,3,1,0,0,marketing,medium,0.83,0.67,0.428571,0.551613,0.3,0.5,0.555556,0.0
4166,0.70,0.74,3,157,3,0,0,0,technical,high,0.74,0.70,0.428571,0.506452,0.3,1.0,1.000000,0.0
1753,0.91,1.00,4,245,6,0,1,0,RandD,medium,1.00,0.91,0.571429,0.790323,0.6,0.5,0.111111,1.0
3775,0.70,0.71,5,269,3,0,0,0,technical,low,0.71,0.70,0.714286,0.867742,0.3,0.0,1.000000,0.0
2998,0.53,0.53,4,205,3,0,0,0,sales,low,0.53,0.53,0.571429,0.661290,0.3,0.0,0.777778,0.0
10169,0.15,0.56,4,214,5,0,0,0,sales,high,0.56,0.15,0.571429,0.690323,0.5,1.0,0.777778,0.0
13869,0.66,0.78,4,161,6,0,0,0,IT,low,0.78,0.66,0.571429,0.519355,0.6,0.0,0.000000,0.0
12762,0.41,0.47,2,131,3,0,1,0,marketing,medium,0.47,0.41,0.285714,0.422581,0.3,0.5,0.555556,1.0
7474,0.66,0.63,3,166,3,0,0,0,IT,high,0.63,0.66,0.428571,0.535484,0.3,1.0,0.000000,0.0
10049,0.25,0.80,5,237,6,0,0,0,support,medium,0.80,0.25,0.714286,0.764516,0.6,0.5,0.888889,0.0


###### Implement PCA to identify correlations

In [10]:
all_features = ['last_evaluation_norm','satisfaction_level_norm','number_project_norm','average_montly_hours_norm',
     'time_spend_company_norm','salary_norm','Work_accident','promotion_last_5years','dept_norm']
specific_features = ['satisfaction_level_norm','number_project_norm',
                      'time_spend_company_norm','salary_norm','Work_accident','promotion_last_5years']
labels = ['left']

In [11]:
# pca_ds = df[all_features].values
# pca = PCA(n_components=2,svd_solver='full')
# pca_y = pca.fit_transform(pca_ds)
# pca_df = pd.DataFrame(pca_y, columns=["PCA1","PCA2"])
# pca_df["label"] = df["left"] 
# #print(pca.explained_variance_ratio_)  
# #print(pca.singular_values_) 
# print("PCA graph for two dimensions")
# g = sns.FacetGrid(pca_df, col="label")
# g = g.map(plt.scatter, "PCA1","PCA2" )
# #0 is stay, 1 is left

In [12]:
def select_interested_columns():
    df_random = df.iloc[np.random.permutation(len(df))]
    return df_random[all_features].values, df_random[labels].values

In [13]:
training_percent = 0.85
validation_percent = 0.15
X,Y = select_interested_columns()
size_of_ds = len(X)
training_limit = int(size_of_ds * training_percent)
validation_limit = int(size_of_ds * (training_percent + validation_percent))
X,Y = X[:size_of_ds],Y[:size_of_ds]

def split(ds, batch_size):
    return np.array(np.array_split(ds, batch_size))

def get_training_data(batch_size):
    return split(X[: training_limit], batch_size),split(Y[: training_limit], batch_size)

def get_testing_data():
    return X[training_limit:],Y[training_limit:]


###### Organize dataset into training validation and testing sets

In [14]:
x_train,y_train = get_training_data(1000)
x_test, y_test = get_testing_data()
#print(x_train[0])
#print(y_train[0])

###### Get Basic Network Inputs

In [15]:
def neural_net_input(features_size):
    return tf.placeholder(tf.float32, shape=[None, features_size])


def neural_net_label_input(output_size = 1):
    return tf.placeholder(tf.float32, shape=[None,output_size])

def neural_net_keep_prob_input():
    return tf.placeholder(tf.float32, name="keep_prob")

def get_weights_bias(input_num_units, hidden_num_units, hidden_num_units1,output_num_units):
    weights = {
        'hidden': tf.Variable(tf.random_normal([input_num_units, hidden_num_units])),
        'hidden1': tf.Variable(tf.random_normal([hidden_num_units, hidden_num_units1])),
        'output': tf.Variable(tf.random_normal([hidden_num_units1, output_num_units]))
    }

    biases = {
        'hidden': tf.Variable(tf.zeros(hidden_num_units)),
        'hidden1': tf.Variable(tf.zeros(hidden_num_units1)),
        'output': tf.Variable(tf.zeros(output_num_units))
    }
    return weights,biases

###### Define Network Construction

In [16]:
def churn_net(x, keep_prob, weights, biases):
    hidden_layer = tf.matmul(x, weights['hidden']) +  biases['hidden']
    hidden_layer = tf.nn.relu(hidden_layer)
    hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
    hidden_layer1 = tf.matmul(hidden_layer, weights['hidden1']) +  biases['hidden1']
    hidden_layer1 = tf.nn.relu(hidden_layer1)
    hidden_layer1 = tf.nn.dropout(hidden_layer1, keep_prob)
    output_layer = tf.matmul(hidden_layer1, weights['output']) + biases['output']
    prediction = tf.nn.sigmoid(output_layer)
    return output_layer,prediction

###### Define Hyper Parameters, Cost and Optimizer

In [17]:
tf.reset_default_graph()

# Inputs, Hyper Parameters
number_of_features = 9
number_of_hidden = 5
number_of_hidden1 = 3
number_of_outputs = 1
x = neural_net_input(number_of_features)
y = neural_net_label_input(number_of_outputs)
keep_prob = neural_net_keep_prob_input()
learning_rate = 0.01
prob = 1

# Model
weights, biases = get_weights_bias(number_of_features, number_of_hidden, number_of_hidden1,number_of_outputs)
output_layer,prediction = churn_net(x, keep_prob,weights, biases)
# Loss and Optimizer
mse = tf.square((prediction - y))
cost = tf.reduce_mean(mse)
mse_summary = tf.summary.scalar('MSE', cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


#Init all the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

###### Sample Train Set

In [18]:
# with tf.Session() as sess:
#     sess.run(init)
#     x_train, x_val, x_test, y_train, y_val, y_test = get_batch_data(512)
#     _, loss = sess.run([optimizer, cost], feed_dict = {x: x_train[0], y: y_train[0], keep_prob: prob})
#     print("Orginal Inputs")
#     print(x_train[0][:10])
# #     print("Input Layer")
# #     print(sess.run(hidden_layer, feed_dict = {x: x_train[0], y: y_train[0], keep_prob: prob})[:10])
# #     print("Hidden Layer")
    
# #     print(sess.run(hidden_layer1, feed_dict = {x: x_train[0], y: y_train[0], keep_prob: prob})[:10])
#     print("output_layer")
#     print(sess.run(output_layer, feed_dict = {x: x_train[0], y: y_train[0], keep_prob: prob})[:10])
#     print("sigmoid prediction")
#     print(sess.run(tf.round(prediction), feed_dict = {x: x_train[0], y: y_train[0], keep_prob: prob})[:10])
#     print("Weights")
#     print(sess.run(weights))
#     print("Bias")
#     print(sess.run(biases))
#     print("Total_cost:", loss)
#     print("actual_output")
#     print(y_train[0][:10])

###### Train the neural net with validation

In [19]:
def train_neural_net(epochs=20, batches=512):
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(epochs):
            print("Running Epoch:{0} with Batches:{1}".format(epoch, batches))
            x_train, y_train = get_training_data(batches)
            avg_cost = 0.0
            for batch_number in range(batches):
                _, loss,mse = sess.run([optimizer, cost, mse_summary], feed_dict = {x: x_train[batch_number], y: y_train[batch_number], keep_prob: prob})
                avg_cost += loss/batches
                file_writer.add_summary(mse,epoch* batches +batch_number)
            print("Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost))
            print("Saving the model")
            save_path = saver.save(sess, "model.ckpt")
        #Testing the model accuracy
        x_val,y_val = get_testing_data()
        pred_temp = tf.equal(tf.greater(prediction,0.5), tf.equal(y,1.0))
        accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
        print("TEST ACCURACY:", accuracy.eval({x: x_val, y: y_val,keep_prob: prob}) * 100,"%")
        difference = prediction - y
        se = tf.square(difference)
        val_cost = tf.reduce_mean(se)
        print("TEST COST:", val_cost.eval({x: x_val, y: y_val,keep_prob: prob}))
        file_writer.close()
        

In [20]:
train_neural_net(1000,512)

Running Epoch:0 with Batches:512
Epoch: 1 cost = 0.20686
Saving the model
Running Epoch:1 with Batches:512
Epoch: 2 cost = 0.18845
Saving the model
Running Epoch:2 with Batches:512
Epoch: 3 cost = 0.18483
Saving the model
Running Epoch:3 with Batches:512
Epoch: 4 cost = 0.18270
Saving the model
Running Epoch:4 with Batches:512
Epoch: 5 cost = 0.17995
Saving the model
Running Epoch:5 with Batches:512
Epoch: 6 cost = 0.17597
Saving the model
Running Epoch:6 with Batches:512
Epoch: 7 cost = 0.17063
Saving the model
Running Epoch:7 with Batches:512
Epoch: 8 cost = 0.16479
Saving the model
Running Epoch:8 with Batches:512
Epoch: 9 cost = 0.15953
Saving the model
Running Epoch:9 with Batches:512
Epoch: 10 cost = 0.15541
Saving the model
Running Epoch:10 with Batches:512
Epoch: 11 cost = 0.15254
Saving the model
Running Epoch:11 with Batches:512
Epoch: 12 cost = 0.15050
Saving the model
Running Epoch:12 with Batches:512
Epoch: 13 cost = 0.14890
Saving the model
Running Epoch:13 with Batches:5

Epoch: 109 cost = 0.09772
Saving the model
Running Epoch:109 with Batches:512
Epoch: 110 cost = 0.09754
Saving the model
Running Epoch:110 with Batches:512
Epoch: 111 cost = 0.09736
Saving the model
Running Epoch:111 with Batches:512
Epoch: 112 cost = 0.09719
Saving the model
Running Epoch:112 with Batches:512
Epoch: 113 cost = 0.09702
Saving the model
Running Epoch:113 with Batches:512
Epoch: 114 cost = 0.09686
Saving the model
Running Epoch:114 with Batches:512
Epoch: 115 cost = 0.09670
Saving the model
Running Epoch:115 with Batches:512
Epoch: 116 cost = 0.09655
Saving the model
Running Epoch:116 with Batches:512
Epoch: 117 cost = 0.09640
Saving the model
Running Epoch:117 with Batches:512
Epoch: 118 cost = 0.09625
Saving the model
Running Epoch:118 with Batches:512
Epoch: 119 cost = 0.09611
Saving the model
Running Epoch:119 with Batches:512
Epoch: 120 cost = 0.09597
Saving the model
Running Epoch:120 with Batches:512
Epoch: 121 cost = 0.09584
Saving the model
Running Epoch:121 wit

Epoch: 215 cost = 0.09053
Saving the model
Running Epoch:215 with Batches:512
Epoch: 216 cost = 0.09051
Saving the model
Running Epoch:216 with Batches:512
Epoch: 217 cost = 0.09049
Saving the model
Running Epoch:217 with Batches:512
Epoch: 218 cost = 0.09046
Saving the model
Running Epoch:218 with Batches:512
Epoch: 219 cost = 0.09044
Saving the model
Running Epoch:219 with Batches:512
Epoch: 220 cost = 0.09042
Saving the model
Running Epoch:220 with Batches:512
Epoch: 221 cost = 0.09040
Saving the model
Running Epoch:221 with Batches:512
Epoch: 222 cost = 0.09038
Saving the model
Running Epoch:222 with Batches:512
Epoch: 223 cost = 0.09036
Saving the model
Running Epoch:223 with Batches:512
Epoch: 224 cost = 0.09034
Saving the model
Running Epoch:224 with Batches:512
Epoch: 225 cost = 0.09032
Saving the model
Running Epoch:225 with Batches:512
Epoch: 226 cost = 0.09030
Saving the model
Running Epoch:226 with Batches:512
Epoch: 227 cost = 0.09028
Saving the model
Running Epoch:227 wit

Epoch: 321 cost = 0.08906
Saving the model
Running Epoch:321 with Batches:512
Epoch: 322 cost = 0.08905
Saving the model
Running Epoch:322 with Batches:512
Epoch: 323 cost = 0.08904
Saving the model
Running Epoch:323 with Batches:512
Epoch: 324 cost = 0.08903
Saving the model
Running Epoch:324 with Batches:512
Epoch: 325 cost = 0.08902
Saving the model
Running Epoch:325 with Batches:512
Epoch: 326 cost = 0.08901
Saving the model
Running Epoch:326 with Batches:512
Epoch: 327 cost = 0.08900
Saving the model
Running Epoch:327 with Batches:512
Epoch: 328 cost = 0.08900
Saving the model
Running Epoch:328 with Batches:512
Epoch: 329 cost = 0.08899
Saving the model
Running Epoch:329 with Batches:512
Epoch: 330 cost = 0.08898
Saving the model
Running Epoch:330 with Batches:512
Epoch: 331 cost = 0.08897
Saving the model
Running Epoch:331 with Batches:512
Epoch: 332 cost = 0.08896
Saving the model
Running Epoch:332 with Batches:512
Epoch: 333 cost = 0.08895
Saving the model
Running Epoch:333 wit

Epoch: 427 cost = 0.08833
Saving the model
Running Epoch:427 with Batches:512
Epoch: 428 cost = 0.08833
Saving the model
Running Epoch:428 with Batches:512
Epoch: 429 cost = 0.08832
Saving the model
Running Epoch:429 with Batches:512
Epoch: 430 cost = 0.08832
Saving the model
Running Epoch:430 with Batches:512
Epoch: 431 cost = 0.08831
Saving the model
Running Epoch:431 with Batches:512
Epoch: 432 cost = 0.08831
Saving the model
Running Epoch:432 with Batches:512
Epoch: 433 cost = 0.08830
Saving the model
Running Epoch:433 with Batches:512
Epoch: 434 cost = 0.08830
Saving the model
Running Epoch:434 with Batches:512
Epoch: 435 cost = 0.08829
Saving the model
Running Epoch:435 with Batches:512
Epoch: 436 cost = 0.08829
Saving the model
Running Epoch:436 with Batches:512
Epoch: 437 cost = 0.08828
Saving the model
Running Epoch:437 with Batches:512
Epoch: 438 cost = 0.08828
Saving the model
Running Epoch:438 with Batches:512
Epoch: 439 cost = 0.08827
Saving the model
Running Epoch:439 wit

Epoch: 533 cost = 0.08774
Saving the model
Running Epoch:533 with Batches:512
Epoch: 534 cost = 0.08773
Saving the model
Running Epoch:534 with Batches:512
Epoch: 535 cost = 0.08773
Saving the model
Running Epoch:535 with Batches:512
Epoch: 536 cost = 0.08772
Saving the model
Running Epoch:536 with Batches:512
Epoch: 537 cost = 0.08772
Saving the model
Running Epoch:537 with Batches:512
Epoch: 538 cost = 0.08771
Saving the model
Running Epoch:538 with Batches:512
Epoch: 539 cost = 0.08771
Saving the model
Running Epoch:539 with Batches:512
Epoch: 540 cost = 0.08771
Saving the model
Running Epoch:540 with Batches:512
Epoch: 541 cost = 0.08770
Saving the model
Running Epoch:541 with Batches:512
Epoch: 542 cost = 0.08770
Saving the model
Running Epoch:542 with Batches:512
Epoch: 543 cost = 0.08769
Saving the model
Running Epoch:543 with Batches:512
Epoch: 544 cost = 0.08769
Saving the model
Running Epoch:544 with Batches:512
Epoch: 545 cost = 0.08768
Saving the model
Running Epoch:545 wit

Epoch: 639 cost = 0.08731
Saving the model
Running Epoch:639 with Batches:512
Epoch: 640 cost = 0.08731
Saving the model
Running Epoch:640 with Batches:512
Epoch: 641 cost = 0.08731
Saving the model
Running Epoch:641 with Batches:512
Epoch: 642 cost = 0.08730
Saving the model
Running Epoch:642 with Batches:512
Epoch: 643 cost = 0.08730
Saving the model
Running Epoch:643 with Batches:512
Epoch: 644 cost = 0.08730
Saving the model
Running Epoch:644 with Batches:512
Epoch: 645 cost = 0.08729
Saving the model
Running Epoch:645 with Batches:512
Epoch: 646 cost = 0.08729
Saving the model
Running Epoch:646 with Batches:512
Epoch: 647 cost = 0.08729
Saving the model
Running Epoch:647 with Batches:512
Epoch: 648 cost = 0.08728
Saving the model
Running Epoch:648 with Batches:512
Epoch: 649 cost = 0.08728
Saving the model
Running Epoch:649 with Batches:512
Epoch: 650 cost = 0.08728
Saving the model
Running Epoch:650 with Batches:512
Epoch: 651 cost = 0.08728
Saving the model
Running Epoch:651 wit

Running Epoch:744 with Batches:512
Epoch: 745 cost = 0.08704
Saving the model
Running Epoch:745 with Batches:512
Epoch: 746 cost = 0.08704
Saving the model
Running Epoch:746 with Batches:512
Epoch: 747 cost = 0.08704
Saving the model
Running Epoch:747 with Batches:512
Epoch: 748 cost = 0.08704
Saving the model
Running Epoch:748 with Batches:512
Epoch: 749 cost = 0.08704
Saving the model
Running Epoch:749 with Batches:512
Epoch: 750 cost = 0.08703
Saving the model
Running Epoch:750 with Batches:512
Epoch: 751 cost = 0.08703
Saving the model
Running Epoch:751 with Batches:512
Epoch: 752 cost = 0.08703
Saving the model
Running Epoch:752 with Batches:512
Epoch: 753 cost = 0.08703
Saving the model
Running Epoch:753 with Batches:512
Epoch: 754 cost = 0.08703
Saving the model
Running Epoch:754 with Batches:512
Epoch: 755 cost = 0.08702
Saving the model
Running Epoch:755 with Batches:512
Epoch: 756 cost = 0.08702
Saving the model
Running Epoch:756 with Batches:512
Epoch: 757 cost = 0.08702
Sav

Epoch: 850 cost = 0.08685
Saving the model
Running Epoch:850 with Batches:512
Epoch: 851 cost = 0.08685
Saving the model
Running Epoch:851 with Batches:512
Epoch: 852 cost = 0.08684
Saving the model
Running Epoch:852 with Batches:512
Epoch: 853 cost = 0.08684
Saving the model
Running Epoch:853 with Batches:512
Epoch: 854 cost = 0.08684
Saving the model
Running Epoch:854 with Batches:512
Epoch: 855 cost = 0.08684
Saving the model
Running Epoch:855 with Batches:512
Epoch: 856 cost = 0.08684
Saving the model
Running Epoch:856 with Batches:512
Epoch: 857 cost = 0.08684
Saving the model
Running Epoch:857 with Batches:512
Epoch: 858 cost = 0.08683
Saving the model
Running Epoch:858 with Batches:512
Epoch: 859 cost = 0.08683
Saving the model
Running Epoch:859 with Batches:512
Epoch: 860 cost = 0.08683
Saving the model
Running Epoch:860 with Batches:512
Epoch: 861 cost = 0.08683
Saving the model
Running Epoch:861 with Batches:512
Epoch: 862 cost = 0.08683
Saving the model
Running Epoch:862 wit

Epoch: 956 cost = 0.08668
Saving the model
Running Epoch:956 with Batches:512
Epoch: 957 cost = 0.08668
Saving the model
Running Epoch:957 with Batches:512
Epoch: 958 cost = 0.08668
Saving the model
Running Epoch:958 with Batches:512
Epoch: 959 cost = 0.08668
Saving the model
Running Epoch:959 with Batches:512
Epoch: 960 cost = 0.08668
Saving the model
Running Epoch:960 with Batches:512
Epoch: 961 cost = 0.08668
Saving the model
Running Epoch:961 with Batches:512
Epoch: 962 cost = 0.08667
Saving the model
Running Epoch:962 with Batches:512
Epoch: 963 cost = 0.08667
Saving the model
Running Epoch:963 with Batches:512
Epoch: 964 cost = 0.08667
Saving the model
Running Epoch:964 with Batches:512
Epoch: 965 cost = 0.08667
Saving the model
Running Epoch:965 with Batches:512
Epoch: 966 cost = 0.08667
Saving the model
Running Epoch:966 with Batches:512
Epoch: 967 cost = 0.08667
Saving the model
Running Epoch:967 with Batches:512
Epoch: 968 cost = 0.08667
Saving the model
Running Epoch:968 wit

###### Loading the model and caluculation Conf

In [21]:
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model.ckpt")
    print("Model restored.")
    x_test, y_test = get_testing_data()
    rounded_pred = tf.round(prediction)
    pred_result = rounded_pred.eval({x: x_test, y: y_test,keep_prob: prob})
    pred_actual = zip(pred_result.ravel(),y_test.ravel())
    #print([x for x in pred_actual])
    print("Precision, Recall, F-Score",precision_recall_fscore_support(y_test.ravel(),pred_result.ravel(), average="binary"))
    #Testing the model accuracy
    pred_temp = tf.equal(tf.greater(prediction,0.5), tf.equal(y,1.0))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    print("TEST ACCURACY:", accuracy.eval({x: x_test, y: y_test,keep_prob: prob}) * 100,"%")
    difference = prediction - y
    se = tf.square(difference)
    val_cost = tf.reduce_mean(se)
    print("TEST COST:", val_cost.eval({x: x_test, y: y_test,keep_prob: prob}))
    
  # Check the values of the variables

INFO:tensorflow:Restoring parameters from model.ckpt
Model restored.
Precision, Recall, F-Score (0.92788461538461542, 0.69175627240143367, 0.79260780287474331, None)
TEST ACCURACY: 91.0222232342 %
TEST COST: 0.0878961


###### Manual Test 

In [35]:
print(all_features)
manual_test_x = [[0.85,0.2,0.3,0.8,0.2,0.3,0.0,0.0,0.5]]
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model.ckpt")
    print("Model restored.")
    rounded_pred = tf.round(prediction)
    pred_result = rounded_pred.eval({x: manual_test_x,keep_prob: 1.0})
    print(pred_result)

['last_evaluation_norm', 'satisfaction_level_norm', 'number_project_norm', 'average_montly_hours_norm', 'time_spend_company_norm', 'salary_norm', 'Work_accident', 'promotion_last_5years', 'dept_norm']
INFO:tensorflow:Restoring parameters from model.ckpt
Model restored.
[[ 0.]]


In [96]:
# sess.run(init)
# print(sess.run(biases))
# # x_train, x_val, x_test, y_train, y_val, y_test = get_batch_data(1)
# # pred_temp = tf.equal(tf.greater(prediction,0.5), tf.equal(y,1.0))
# # accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
# # print("FINAL ACCURACY:", accuracy.eval({x: x_test, y: y_test,keep_prob: prob}) * 100,"%")